<a href="https://colab.research.google.com/github/LH-1169213/Equipo3/blob/main/SVD_Equipo09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 10**
###**Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Nombres y matrículas de los integrantes del equipo:**

*   Samara García González A01273001
*   Laura Elena Hernández Mata A01169213
*   Evelyn Aylin Rendon Medina A01748750
*   Emmanuel González Calitl A01320739
*   Carlos Enrique Diaz Ayala A01793816


En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [1]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD

#**Ejercicio-1.** 

In [2]:
#rating_final.csv
df_ratingfinal = pd.read_csv("https://raw.githubusercontent.com/LH-1169213/Equipo3/main/rating_final.csv")
df_ratingfinal.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [3]:
#Obteniendo datos de valor del archivo rating_final.csv
UserID = np.array(df_ratingfinal[['userID']]).flatten()
print(UserID.shape)
rating = np.array(df_ratingfinal[['rating']]).flatten()
print(rating.shape)

(1161,)
(1161,)


In [4]:
# u'\xe9'.encode('latin-1')

#geoplaces2.csv
df_geoplaces = pd.read_csv("https://raw.githubusercontent.com/LH-1169213/Equipo3/main/geoplaces2.csv", encoding = "latin-1")
df_geoplaces.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none


In [5]:
# Creamos un diccionario con llave (placeID) y sus nombres
place_map = pd.Series(df_geoplaces.name.values, index = df_geoplaces.placeID).to_dict()
place_map

{134999: 'Kiku Cuernavaca',
 132825: 'puesto de tacos',
 135106: 'El Rincón de San Francisco',
 132667: 'little pizza Emilio Portes Gil',
 132613: 'carnitas_mata',
 135040: 'Restaurant los Compadres',
 132732: 'Taqueria EL amigo ',
 132875: 'shi ro ie',
 132609: 'Pollo_Frito_Buenos_Aires',
 135082: 'la Estrella de Dimas',
 135070: 'Restaurante 75',
 135069: 'Abondance Restaurante Bar',
 135065: 'El angel Restaurante',
 135076: 'Restaurante Pueblo Bonito',
 135086: 'Mcdonalds Parque Tangamanga',
 132870: 'Tortas y hamburguesas el gordo',
 132854: 'Sirlone',
 132937: 'rockabilly ',
 132856: 'Unicols Pizza',
 132668: 'TACOS EL GUERO',
 135033: 'Restaurant El Muladar de Calzada',
 132862: 'La Posada del Virrey',
 134983: 'Restaurant and Bar and Clothesline Carlos N Charlies',
 132851: 'KFC',
 135013: 'Giovannis',
 135042: 'Restaurant Oriental Express',
 132768: 'Mariscos Tia Licha',
 132561: 'cafe ambar',
 135045: 'Restaurante la Gran Via',
 132847: 'don burguers',
 135054: 'Restaurante y 

In [6]:
# Creamos un DF con los valores previamente obtenidos de las dos BD

df = pd.DataFrame({"UserID": UserID,"rating": rating})
df.head()
df.shape


(1161, 2)

In [7]:
# Agregamos el nombre del lugar que están calificando los usuarios

df["place"] = df_ratingfinal['placeID'].map(place_map)
df.head()

,UserID,rating,place
0,U1077,2,Tortas Locas Hipocampo
1,U1077,2,Restaurant la Chalita
2,U1077,2,puesto de tacos
3,U1077,1,Restaurante Marisco Sam
4,U1068,1,vips


In [8]:
#Matriz de utilidad
matriz = df.pivot_table(values='rating', index='UserID', columns='place', fill_value=0)
print('Tamaño de la matriz de utilidad:', matriz.shape)
print('número de filas:',matriz.shape[0])
print('número de columnas:',matriz.shape[1])

Tamaño de la matriz de utilidad: (138, 129)
número de filas: 138
número de columnas: 129


In [9]:
matriz.head(10)

place,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
UserID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [10]:
print('Total de elementos de la matriz de utilidad: %d' % (matriz.size))
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(matriz)))
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(matriz) / matriz.size))

Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 907
Porcentaje de elementos diferentes de cero: 5.1%


#**Ejercicio-2.**

In [11]:
#Transpuesta de la matriz de utilidad
#Para obtener la información latente en relación a los lugares de comida
matrizT = matriz.T
print('Tamaño de la matriz de utilidad:', matriz.shape)
print('número de filas:',matrizT.shape[0])
print('número de columnas:',matrizT.shape[1])

Tamaño de la matriz de utilidad: (138, 129)
número de filas: 129
número de columnas: 138


In [12]:
matrizT.head()

UserID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
place,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,1,0,...,0,0,0,0,0,2,0,0,0,0
Cafe Chaires,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#El número de componentes debe ser menor que el número de características
SVD = TruncatedSVD(n_components=137, random_state=1)  
SVD.fit(matrizT)

num_sv = 5

print('Cantidad de información simplificada con los %d vectores singulares obtenidos:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD.singular_values_[0:num_sv]).sum() / (SVD.singular_values_).sum())))

Cantidad de información simplificada con los 5 vectores singulares obtenidos:
85.5%


#**Ejercicio-3.**

In [107]:
# ENTRENAMOS AL MODELO
New_matrix = SVD.fit_transform(matrizT)
corrMx = np.corrcoef(New_matrix)
print("Shape:")
print(corrMx.shape)

#OBTENEMOS EL ID DEL RESTAURANTE

sugerencia_ej3 = "Subway"

names = matriz.columns
names_list = list(names)
id_restau = names_list.index(sugerencia_ej3)

print("\nID restaurante:")
print(id_restau)

# OBTENEMOS LAS RECOMENDACIONES ORDENADAS DE MAYOR A MENOR
recomienda_ej3 = np.sort(corrMx[id_restau])[::-1][0:12]
resultado_ej3 = recomienda_ej3[-1]
listado_ej3 = list(names[(corrMx[id_restau] > resultado_ej3) & (corrMx[id_restau] < 0.99)])[:11]

# IMPRIMIMOS LOS RESULTADOS
print("\nListado de recomendaciones:")
for i in range(0, len(listado_ej3)):
  print(listado_ej3[i])

Shape:
(129, 129)

ID restaurante:
88

Listado de recomendaciones:
Chilis Cuernavaca
Log Yin
Mariscos Tia Licha
Mikasa
Restaurant Teely
Restaurant and Bar and Clothesline Carlos N Charlies
Restaurant los Pinos
Rincon del Bife
Sanborns Casa Piedra
Vips


In [122]:
# OBTENEMOS LAS CORRELACIONES
corr_final = []
for i in listado_ej3:
  corr_final.extend((np.sort(corrMx[id_restau])[::-1][1:12]))
corr_final

print("\nListado de recomendaciones y su correlación")
for i in range(0, len(listado_ej3)):
  print(listado_ej3[i], corr_final[i]*100,"%")


Listado de recomendaciones y su correlación
Chilis Cuernavaca 67.50589128381256 %
Log Yin 51.2140936999518 %
Mariscos Tia Licha 44.25225431352351 %
Mikasa 44.05536567024261 %
Restaurant Teely 42.020111137832714 %
Restaurant and Bar and Clothesline Carlos N Charlies 38.700232665405125 %
Restaurant los Pinos 36.887172707202026 %
Rincon del Bife 31.582630067040828 %
Sanborns Casa Piedra 31.081593518563942 %
Vips 23.518866571136595 %


#**Ejercicio-4.**

In [125]:
# REALIZAMOS EL EJERCICIO CON LA REDUCCIÓN DE COMPONENTES
SVD_ej4 = TruncatedSVD(n_components=60, random_state=1)  
SVD_ej4.fit(matrizT)

num_sv = 5

print('Cantidad de información simplificada con los %d vectores singulares obtenidos:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD_ej4.singular_values_[0:num_sv]).sum() / (SVD_ej4.singular_values_).sum())))

# ENTRENAMOS AL MODELO
Matrix_ej4 = SVD_ej4.fit_transform(matrizT)
corrMx_ej4 = np.corrcoef(Matrix_ej4)
corrMx_ej4.shape

#OBTENEMOS EL ID DEL RESTAURANTE
sugerencia_ej4 = "Subway"

names_ej4 = matriz.columns
names_list_ej4 = list(names_ej4)
id_restau_4 = names_list_ej4.index(sugerencia_ej4)

print("\nID restaurante:")
print(id_restau_4)

# OBTENEMOS LAS RECOMENDACIONES ORDENADAS DE MAYOR A MENOR
recomienda_ej4 = np.sort(corrMx_ej4[id_restau_4])[::-1][0:12]
resultado_ej4 = recomienda_ej4[-1]
listado_ej4 = list(names[(corrMx_ej4[id_restau_4] > resultado_ej4) & (corrMx_ej4[id_restau_4] < 0.99)])[:11]

# IMPRIMIMOS LOS RESULTADOS
print("\nListado de recomendaciones:")
for i in range(0, len(listado_ej4)):
  print(listado_ej4[i])

Cantidad de información simplificada con los 5 vectores singulares obtenidos:
81.8%

ID restaurante:
88

Listado de recomendaciones:
Chilis Cuernavaca
Mariscos Tia Licha
Mikasa
Paniroles
Restaurant Teely
Restaurant and Bar and Clothesline Carlos N Charlies
Restaurant los Pinos
Rincon del Bife
Sanborns Casa Piedra
Vips


In [127]:
# OBTENEMOS LAS CORRELACIONES
corr_final_ej4 = []
for i in listado_ej4:
  corr_final_ej4.extend((np.sort(corrMx_ej4[id_restau_4])[::-1][1:12]))
corr_final_ej4

print("\nListado de recomendaciones y su correlación")
for i in range(0, len(listado_ej4)):
  print(listado_ej4[i], corr_final_ej4[i]*100,"%")


Listado de recomendaciones y su correlación
Chilis Cuernavaca 77.7641581511475 %
Mariscos Tia Licha 69.61823043340023 %
Mikasa 59.739657474784956 %
Paniroles 53.01455726172435 %
Restaurant Teely 50.02655525709859 %
Restaurant and Bar and Clothesline Carlos N Charlies 45.014639174003825 %
Restaurant los Pinos 42.176904301083816 %
Rincon del Bife 34.27410296064025 %
Sanborns Casa Piedra 29.709830131499277 %
Vips 24.040946246677798 %


#**Ejercicio-5.**

In [129]:
# CALCULAMOS CON LA TOTALIDAD DE LA INFORMACIÓN

SVD_ej5 = TruncatedSVD(n_components=137, random_state=1)  
SVD_ej5.fit(matrizT)

num_sv = 5

print('Cantidad de información simplificada con los %d vectores singulares obtenidos:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD_ej5.singular_values_[0:num_sv]).sum() / (SVD_ej5.singular_values_).sum())))

Matrix_ej5 = SVD_ej5.fit_transform(matrizT)
corrMx_ej5 = np.corrcoef(Matrix_ej5)
print("\n Shape:")
print(corrMx_ej5.shape)

#OBTENEMOS EL ID DEL RESTAURANTE
sugerencia_ej5 = "tacos de barbacoa enfrente del Tec"

names_ej5 = matriz.columns
names_list_ej5 = list(names_ej5)
id_restau_5 = names_list_ej5.index(sugerencia_ej5)

print("\nID Restaurante:")
print(id_restau_5)

# OBTENEMOS LAS RECOMENDACIONES ORDENADAS DE MAYOR A MENOR
recomienda_ej5 = np.sort(corrMx_ej5[id_restau_5])[::-1][0:12]
resultado_ej5 = recomienda_ej5[-1]
listado_ej5 = list(names[(corrMx_ej5[id_restau_5] > resultado_ej5) & (corrMx_ej5[id_restau_5] < 0.99)])[:11]

# IMPRIMIMOS LOS RESULTADOS
print("\nListado de recomendaciones:")
for i in range(0, len(listado_ej5)):
  print(listado_ej5[i])

Cantidad de información simplificada con los 5 vectores singulares obtenidos:
85.5%

 Shape:
(129, 129)

ID Restaurante:
124

Listado de recomendaciones:
Carreton de Flautas y Migadas
Hamburguesas La perica
Little Cesarz
Taqueria EL amigo 
carnitas mata calle Emilio Portes Gil
little pizza Emilio Portes Gil
palomo tec
puesto de gorditas
tacos abi
vips


In [130]:
# OBTENEMOS LAS CORRELACIONES
corr_final_ej5 = []
for i in listado_ej5:
  corr_final_ej5.extend((np.sort(corrMx_ej5[id_restau_5])[::-1][1:12]))
corr_final_ej5

print("\nListado de recomendaciones y su correlación")
for i in range(0, len(listado_ej5)):
  print(listado_ej5[i], corr_final_ej5[i]*100,"%")


Listado de recomendaciones y su correlación
Carreton de Flautas y Migadas 81.71889849105266 %
Hamburguesas La perica 66.53053654632711 %
Little Cesarz 65.38492684416552 %
Taqueria EL amigo  43.7184208426359 %
carnitas mata calle Emilio Portes Gil 41.276452318483905 %
little pizza Emilio Portes Gil 40.7794920181297 %
palomo tec 25.847938542625933 %
puesto de gorditas 19.51740880107841 %
tacos abi 19.112045068235023 %
vips 17.093947240845456 %


In [132]:
# CALCULAMOS CON UNA PARTE DE LA INFORMACIÓN
SVD_ej5_V2 = TruncatedSVD(n_components=80, random_state=1)  
SVD_ej5_V2.fit(matrizT)

num_sv = 5

print('Cantidad de información simplificada con los %d vectores singulares obtenidos:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD_ej5_V2.singular_values_[0:num_sv]).sum() / (SVD_ej5_V2.singular_values_).sum())))

# ENTRENAMOS AL MODELO
Matrix_ej5_V2 = SVD_ej5_V2.fit_transform(matrizT)
corrMx_ej5_V2 = np.corrcoef(Matrix_ej5_V2)
print("\n Shape:")
print(corrMx_ej5_V2.shape)

#OBTENEMOS EL ID DEL RESTAURANTE
sugerencia_ej5_V2 = "tacos de barbacoa enfrente del Tec"

names_ej5_V2 = matriz.columns
names_list_ej5_V2 = list(names_ej5_V2)
id_restau_5_V2 = names_list_ej5_V2.index(sugerencia_ej5_V2)

print("\n ID Restaurante:")
print(id_restau_5_V2)

# OBTENEMOS LAS RECOMENDACIONES ORDENADAS DE MAYOR A MENOR
recomienda_ej5_V2 = np.sort(corrMx[id_restau_5_V2])[::-1][0:12]
resultado_ej5_V2 = recomienda_ej5_V2[-1]
listado_ej5_V2 = list(names[(corrMx_ej5_V2[id_restau_5_V2] > resultado_ej5_V2) & (corrMx_ej5_V2[id_restau_5_V2] < 0.99)])[:11]

# IMPRIMIMOS LOS RESULTADOS
print("\nListado de recomendaciones:")
for i in range(0, len(listado_ej5_V2)):
  print(listado_ej5_V2[i])

Cantidad de información simplificada con los 5 vectores singulares obtenidos:
84.0%

 Shape:
(129, 129)

 ID Restaurante:
124

Listado de recomendaciones:
Carreton de Flautas y Migadas
Hamburguesas La perica
Little Cesarz
Taqueria EL amigo 
carnitas mata calle Emilio Portes Gil
carnitas_mata
little pizza Emilio Portes Gil
puesto de gorditas
tacos abi
vips


In [133]:
# OBTENEMOS LAS CORRELACIONES
corr_final_ej5_V2 = []
for i in listado_ej5_V2:
  corr_final_ej5_V2.extend((np.sort(corrMx_ej5_V2[id_restau_5_V2])[::-1][1:12]))
corr_final_ej5_V2

print("\nListado de recomendaciones y su correlación")
for i in range(0, len(listado_ej5_V2)):
  print(listado_ej5_V2[i], corr_final_ej5_V2[i]*100,"%")


Listado de recomendaciones y su correlación
Carreton de Flautas y Migadas 86.55982233473783 %
Hamburguesas La perica 85.81509913706923 %
Little Cesarz 84.51630975384651 %
Taqueria EL amigo  57.06558654856195 %
carnitas mata calle Emilio Portes Gil 52.11317553851258 %
carnitas_mata 50.685600887632155 %
little pizza Emilio Portes Gil 31.56593588458382 %
puesto de gorditas 26.56445981651338 %
tacos abi 20.298874078229854 %
vips 17.796731760012992 %


# **Incluye tus conclusiones**

#**Ejercicio-6.**

In [27]:
#Obteniendo datos de valor del archivo rating_final.csv
UserID = np.array(df_ratingfinal[['userID']]).flatten()
print(UserID.shape)
food_rating = np.array(df_ratingfinal[['food_rating']]).flatten()
print(food_rating.shape)

(1161,)
(1161,)


In [29]:
# Creamos un DF con los valores previamente obtenidos de las dos BD

df_food = pd.DataFrame({"UserID": UserID,"food_rating": food_rating})
df_food.head()
df_food.shape

(1161, 2)

In [30]:
# Agregamos el nombre del lugar que están calificando los usuarios

df_food["place"] = df_ratingfinal['placeID'].map(place_map)
df_food.head()

,UserID,food_rating,place
0,U1077,2,Tortas Locas Hipocampo
1,U1077,2,Restaurant la Chalita
2,U1077,2,puesto de tacos
3,U1077,2,Restaurante Marisco Sam
4,U1068,1,vips


In [31]:
#Matriz de utilidad
matriz_food = df_food.pivot_table(values='food_rating', index='UserID', columns='place', fill_value=0)
print('Tamaño de la matriz de utilidad:', matriz_food.shape)
print('número de filas:',matriz_food.shape[0])
print('número de columnas:',matriz_food.shape[1])

Tamaño de la matriz de utilidad: (138, 129)
número de filas: 138
número de columnas: 129


In [33]:
matriz_food.head(10)

place,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
UserID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,0,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,2,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [34]:
print('Total de elementos de la matriz de utilidad: %d' % (matriz_food.size))
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(matriz_food)))
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(matriz_food) / matriz_food.size))

Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 895
Porcentaje de elementos diferentes de cero: 5.0%


In [35]:
#Transpuesta de la matriz de utilidad
#Para obtener la información latente en relación a los lugares de comida
matrizT_food = matriz_food.T
print('Tamaño de la matriz de utilidad:', matriz_food.shape)
print('número de filas:',matrizT_food.shape[0])
print('número de columnas:',matrizT_food.shape[1])

Tamaño de la matriz de utilidad: (138, 129)
número de filas: 129
número de columnas: 138


In [36]:
matrizT_food.head()

UserID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
place,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,2,0,...,0,0,0,0,0,1,0,0,0,0
Cafe Chaires,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [136]:
# REALIZAMOS EL EJERCICIO CON LA TOTALIDAD DE COMPONENTES
# El número de componentes debe ser menor que el número de características
SVD_food = TruncatedSVD(n_components=137, random_state=1)  
SVD_food.fit(matrizT_food)

num_sv = 5

print('Cantidad de información simplificada con los %d vectores singulares obtenidos:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD_food.singular_values_[0:num_sv]).sum() / (SVD_food.singular_values_).sum())))

# ENTRENAMOS AL MODELO
New_matrix_food = SVD_food.fit_transform(matrizT_food)
corrMx_food = np.corrcoef(New_matrix_food)
print("\nShape:")
print(corrMx_food.shape)

#OBTENEMOS EL ID DEL RESTAURANTE

sugerencia_food = "Subway"

names_food = matriz_food.columns
names_list_food = list(names_food)
id_restau_food = names_list.index(sugerencia_food)

print("\nID restaurante:")
print(id_restau_food)

# OBTENEMOS LAS RECOMENDACIONES ORDENADAS DE MAYOR A MENOR
recomienda_food = np.sort(corrMx_food[id_restau_food])[::-1][0:12]
resultado_food = recomienda_food[-1]
listado_food = list(names_food[(corrMx_food[id_restau_food] > resultado_food) & (corrMx_food[id_restau_food] < 0.99)])[:11]

# IMPRIMIMOS LOS RESULTADOS
print("\nListado de recomendaciones:")
for i in range(0, len(listado_food)):
  print(listado_food[i])

Cantidad de información simplificada con los 5 vectores singulares obtenidos:
85.9%

Shape:
(129, 129)

ID restaurante:
88

Listado de recomendaciones:
Kiku Cuernavaca
Mariscos Tia Licha
Paniroles
Restaurant Familiar El Chino
Restaurant Las Mananitas
Restaurant Teely
Restaurant and Bar and Clothesline Carlos N Charlies
Rincon del Bife
Sanborns Casa Piedra
Vips


In [137]:
# OBTENEMOS LAS CORRELACIONES
corr_final_food = []
for i in listado_food:
  corr_final_food.extend((np.sort(corrMx_food[id_restau_food])[::-1][1:12]))
corr_final_food

print("\nListado de recomendaciones y su correlación")
for i in range(0, len(listado_food)):
  print(listado_food[i], corr_final_food[i]*100,"%")


Listado de recomendaciones y su correlación
Kiku Cuernavaca 81.64133737408036 %
Mariscos Tia Licha 46.8949685810932 %
Paniroles 44.21477019057171 %
Restaurant Familiar El Chino 41.52220526777537 %
Restaurant Las Mananitas 38.181946304071396 %
Restaurant Teely 36.5656687999298 %
Restaurant and Bar and Clothesline Carlos N Charlies 33.17067803394996 %
Rincon del Bife 31.2082362074603 %
Sanborns Casa Piedra 25.035740003437574 %
Vips 23.27083683490996 %


In [138]:
# REALIZAMOS EL EJERCICIO CON LA REDUCCIÓN DE COMPONENTES
# El número de componentes debe ser menor que el número de características
SVD_food_v2 = TruncatedSVD(n_components=75, random_state=1)  
SVD_food_v2.fit(matrizT_food)

num_sv = 5

print('Cantidad de información simplificada con los %d vectores singulares obtenidos:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD_food_v2.singular_values_[0:num_sv]).sum() / (SVD_food_v2.singular_values_).sum())))

# ENTRENAMOS AL MODELO
New_matrix_food_v2 = SVD_food_v2.fit_transform(matrizT_food)
corrMx_food_v2 = np.corrcoef(New_matrix_food_v2)
print("\n Shape:")
print(corrMx_food_v2.shape)

#OBTENEMOS EL ID DEL RESTAURANTE

sugerencia_food_v2 = "Subway"

names_food_v2 = matriz_food.columns
names_list_food_v2 = list(names_food_v2)
id_restau_food_v2 = names_list.index(sugerencia_food_v2)

print("\n ID restaurante:")
print(id_restau_food_v2)

# OBTENEMOS LAS RECOMENDACIONES ORDENADAS DE MAYOR A MENOR
recomienda_food_v2 = np.sort(corrMx_food_v2[id_restau_food_v2])[::-1][0:12]
resultado_food_v2 = recomienda_food_v2[-1]
listado_food_v2 = list(names_food_v2[(corrMx_food_v2[id_restau_food_v2] > resultado_food_v2) & (corrMx_food_v2[id_restau_food_v2] < 0.99)])[:11]

# IMPRIMIMOS LOS RESULTADOS
print("\nListado de recomendaciones:")
for i in range(0, len(listado_food_v2)):
  print(listado_food_v2[i])

Cantidad de información simplificada con los 5 vectores singulares obtenidos:
84.0%

 Shape:
(129, 129)

 ID restaurante:
88

Listado de recomendaciones:
Kiku Cuernavaca
Mariscos Tia Licha
Paniroles
Restaurant Familiar El Chino
Restaurant Las Mananitas
Restaurant Teely
Restaurant and Bar and Clothesline Carlos N Charlies
Rincon del Bife
Sanborns Casa Piedra
Vips


In [139]:
# OBTENEMOS LAS CORRELACIONES
corr_final_food_v2 = []
for i in listado_food_v2:
  corr_final_food_v2.extend((np.sort(corrMx_food_v2[id_restau_food_v2])[::-1][1:12]))
corr_final_food_v2

print("\nListado de recomendaciones y su correlación")
for i in range(0, len(listado_food_v2)):
  print(listado_food_v2[i], corr_final_food_v2[i]*100,"%")


Listado de recomendaciones y su correlación
Kiku Cuernavaca 87.06346901624289 %
Mariscos Tia Licha 49.67602310346936 %
Paniroles 45.97920687252975 %
Restaurant Familiar El Chino 44.65780944648725 %
Restaurant Las Mananitas 43.3232381652553 %
Restaurant Teely 38.54608529398668 %
Restaurant and Bar and Clothesline Carlos N Charlies 33.46707793082967 %
Rincon del Bife 31.837836148736987 %
Sanborns Casa Piedra 25.267134179742257 %
Vips 25.08806689498589 %


# **Incluye tus conclusiones**

#**Ejercicio-7.**

In [140]:
#Obteniendo datos de valor del archivo rating_final.csv
UserID = np.array(df_ratingfinal[['userID']]).flatten()
print(UserID.shape)
service_rating = np.array(df_ratingfinal[['service_rating']]).flatten()
print(service_rating.shape)

(1161,)
(1161,)


In [141]:
# Creamos un DF con los valores previamente obtenidos de las dos BD

df_service = pd.DataFrame({"UserID": UserID,"service_rating": service_rating})
df_service.head()
df_service.shape

(1161, 2)

In [142]:
# Agregamos el nombre del lugar que están calificando los usuarios

df_service["place"] = df_ratingfinal['placeID'].map(place_map)
df_service.head()

,UserID,service_rating,place
0,U1077,2,Tortas Locas Hipocampo
1,U1077,1,Restaurant la Chalita
2,U1077,2,puesto de tacos
3,U1077,2,Restaurante Marisco Sam
4,U1068,2,vips


In [143]:
#Matriz de utilidad
matriz_service = df_service.pivot_table(values='service_rating', index='UserID', columns='place', fill_value=0)
print('Tamaño de la matriz de utilidad:', matriz_service.shape)
print('número de filas:',matriz_service.shape[0])
print('número de columnas:',matriz_service.shape[1])

Tamaño de la matriz de utilidad: (138, 129)
número de filas: 138
número de columnas: 129


In [144]:
matriz_service.head(10)

place,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
UserID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [145]:
print('Total de elementos de la matriz de utilidad: %d' % (matriz_service.size))
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(matriz_service)))
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(matriz_service) / matriz_service.size))

Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 846
Porcentaje de elementos diferentes de cero: 4.8%


In [146]:
#Transpuesta de la matriz de utilidad
#Para obtener la información latente en relación a los lugares de comida
matrizT_service = matriz_service.T
print('Tamaño de la matriz de utilidad:', matriz_service.shape)
print('número de filas:',matrizT_service.shape[0])
print('número de columnas:',matrizT_service.shape[1])

Tamaño de la matriz de utilidad: (138, 129)
número de filas: 129
número de columnas: 138


In [147]:
matrizT_service.head()

UserID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
place,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
Cafe Chaires,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [148]:
# REALIZAMOS EL EJERCICIO CON LA TOTALIDAD DE COMPONENTES
# El número de componentes debe ser menor que el número de características
SVD_service = TruncatedSVD(n_components=137, random_state=1)  
SVD_service.fit(matrizT_service)

num_sv = 5

print('Cantidad de información simplificada con los %d vectores singulares obtenidos:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD_service.singular_values_[0:num_sv]).sum() / (SVD_service.singular_values_).sum())))

# ENTRENAMOS AL MODELO
New_matrix_service = SVD_service.fit_transform(matrizT_service)
corrMx_service = np.corrcoef(New_matrix_service)
print("\nShape:")
print(corrMx_service.shape)

#OBTENEMOS EL ID DEL RESTAURANTE

sugerencia_service = "Subway"

names_service = matriz_service.columns
names_list_service = list(names_service)
id_restau_service = names_list_service.index(sugerencia_service)

print("\nID restaurante:")
print(id_restau_service)

# OBTENEMOS LAS RECOMENDACIONES ORDENADAS DE MAYOR A MENOR
recomienda_service = np.sort(corrMx_service[id_restau_service])[::-1][0:12]
resultado_service = recomienda_service[-1]
listado_service = list(names_service[(corrMx_service[id_restau_service] > resultado_service) & (corrMx_food[id_restau_service] < 0.99)])[:11]

# IMPRIMIMOS LOS RESULTADOS
print("\nListado de recomendaciones:")
for i in range(0, len(listado_service)):
  print(listado_service[i])

Cantidad de información simplificada con los 5 vectores singulares obtenidos:
85.6%

Shape:
(129, 129)

ID restaurante:
88

Listado de recomendaciones:
Chilis Cuernavaca
Giovannis
Kiku Cuernavaca
Paniroles
Restaurant Familiar El Chino
Restaurant Las Mananitas
Restaurant Teely
Restaurant and Bar and Clothesline Carlos N Charlies
Rincon del Bife
Vips


In [149]:
# OBTENEMOS LAS CORRELACIONES
corr_final_service = []
for i in listado_service:
  corr_final_service.extend((np.sort(corrMx_service[id_restau_service])[::-1][1:12]))
corr_final_service

print("\nListado de recomendaciones y su correlación")
for i in range(0, len(listado_service)):
  print(listado_service[i], corr_final_service[i]*100,"%")


Listado de recomendaciones y su correlación
Chilis Cuernavaca 77.29731386918377 %
Giovannis 60.01606252966776 %
Kiku Cuernavaca 51.87314706565805 %
Paniroles 42.11998645650441 %
Restaurant Familiar El Chino 36.5933684245345 %
Restaurant Las Mananitas 23.069782868565678 %
Restaurant Teely 22.626624223425203 %
Restaurant and Bar and Clothesline Carlos N Charlies 19.27595939045878 %
Rincon del Bife 18.286947897963053 %
Vips 17.874009403033547 %


In [152]:
# REALIZAMOS EL EJERCICIO CON LA REDUCCIÓN DE COMPONENTES
# El número de componentes debe ser menor que el número de características
SVD_service_v2 = TruncatedSVD(n_components=60, random_state=1)  
SVD_service_v2.fit(matrizT_service)

num_sv = 5

print('Cantidad de información simplificada con los %d vectores singulares obtenidos:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD_service_v2.singular_values_[0:num_sv]).sum() / (SVD_service_v2.singular_values_).sum())))

# ENTRENAMOS AL MODELO
New_matrix_service_v2 = SVD_service_v2.fit_transform(matrizT_service)
corrMx_service_v2 = np.corrcoef(New_matrix_service_v2)
print("\nShape:")
print(corrMx_service_v2.shape)

#OBTENEMOS EL ID DEL RESTAURANTE

sugerencia_service_v2 = "Subway"

names_service_v2 = matriz_service.columns
names_list_service_v2 = list(names_service_v2)
id_restau_service_v2 = names_list_service_v2.index(sugerencia_service_v2)

print("\nID restaurante:")
print(id_restau_service_v2)

# OBTENEMOS LAS RECOMENDACIONES ORDENADAS DE MAYOR A MENOR
recomienda_service_v2 = np.sort(corrMx_service_v2[id_restau_service_v2])[::-1][0:12]
resultado_service_v2 = recomienda_service_v2[-1]
listado_service_v2 = list(names_service_v2[(corrMx_service_v2[id_restau_service_v2] > resultado_service_v2) & (corrMx_food_v2[id_restau_service_v2] < 0.99)])[:11]

# IMPRIMIMOS LOS RESULTADOS
print("\nListado de recomendaciones:")
for i in range(0, len(listado_service_v2)):
  print(listado_service_v2[i])

Cantidad de información simplificada con los 5 vectores singulares obtenidos:
82.1%

Shape:
(129, 129)

ID restaurante:
88

Listado de recomendaciones:
Chilis Cuernavaca
Giovannis
Kiku Cuernavaca
Paniroles
Restaurant Familiar El Chino
Restaurant Las Mananitas
Restaurant Teely
Restaurant and Bar and Clothesline Carlos N Charlies
Rincon del Bife
Vips


In [153]:
# OBTENEMOS LAS CORRELACIONES
corr_final_service_v2 = []
for i in listado_service_v2:
  corr_final_service_v2.extend((np.sort(corrMx_service_v2[id_restau_service_v2])[::-1][1:12]))
corr_final_service_v2

print("\nListado de recomendaciones y su correlación")
for i in range(0, len(listado_service_v2)):
  print(listado_service_v2[i], corr_final_service_v2[i]*100,"%")


Listado de recomendaciones y su correlación
Chilis Cuernavaca 92.6093299623009 %
Giovannis 74.21345742162477 %
Kiku Cuernavaca 67.9960051284857 %
Paniroles 43.53863732807996 %
Restaurant Familiar El Chino 40.10417993057065 %
Restaurant Las Mananitas 25.720902147344297 %
Restaurant Teely 24.80560084419978 %
Restaurant and Bar and Clothesline Carlos N Charlies 22.510969801943183 %
Rincon del Bife 19.31055892892269 %
Vips 18.678764619640205 %


# **Incluye tus conclusiones**

#**Ejercicio-8.**

In [24]:
None

###**Fin de la Actividad de la semana 10 - Sistemas de Recomendación y Reducción de Dimensionalidad con SVD.**